# ⚙️ W04 — Genetic Algorithm Hyperparameter Optimization
**Objective**: Optimize ML and DL branches via two-stage GA (Section III-B4).

**Strategy**:
- Stage 1: GA on XGBoost (fast, ~1h)
- Stage 2: GA on BiLSTM + Attention (slower, ~6-12h)
- Fitness: RMSE on validation set (per-unit, last window)

**Author**: Fatima Khadija Benzine  
**Date**: 23 February 2026

In [1]:
# Colab

In [3]:
# === COLAB SETUP ===
import os

# Clone le repo
if not os.path.exists('/content/PhD-Project-'):
    !git clone https://github.com/f-khadija-benzine/PhD-Project-.git

# Install
!pip install xgboost -q

# Paths
import sys
from pathlib import Path
sys.path.insert(0, '/content/PhD-Project-/src')
project_root = Path('/content/PhD-Project-')

# Check GPU
import tensorflow as tf
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

GPU: []


---
## 0. Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Auto-detect environment
if 'google.colab' in str(globals().get('get_ipython', lambda: '')):
    # Colab: already set by setup cell
    print("Running on Colab ✓")
else:
    # Local
    project_root = Path().resolve().parent
    sys.path.insert(0, str(project_root))
    sys.path.insert(0, str(project_root / 'src'))
    print(f"Running locally ✓ — {project_root}")

from data_loader import MultiDatasetLoader
from preprocessing import PreprocessingPipelineBI, DataNormalizer, create_sliding_windows, evaluate_per_unit
from bi_fusion import BIFusionPipeline, CONTINUOUS_BI_VARS
from feature_selection import BIAwareFeatureSelector
from feature_selection_aficv import AFICvFeatureSelector
from ga_optimizer import run_ml_ga, run_dl_ga

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("All modules imported ✓")

---
## 1. Config & Data Preparation

In [ ]:
# ==================== CONFIG ====================
DATASET = 'FD001'
W = 30
PAD = False
# ================================================

In [ ]:
# Load & common preprocessing
loader = MultiDatasetLoader()
ds = loader.load_cmapss_dataset(DATASET)

meta_cols = ['unit', 'cycle', 'rul']
train_raw = ds['train'].copy()
test_raw = ds['test'].copy()
train_raw['rul'] = train_raw['rul'].clip(upper=125)
if 'rul' in test_raw.columns:
    test_raw['rul'] = test_raw['rul'].clip(upper=125)

sensor_cols = [c for c in train_raw.columns if c.startswith('sensor_')]
setting_cols = [c for c in train_raw.columns if c.startswith('setting_')]

norm = DataNormalizer(method='minmax')
train_norm = norm.fit_transform(train_raw, sensor_cols + setting_cols)
test_norm = norm.transform(test_raw)

fusion = BIFusionPipeline()
train_fused = fusion.fuse(train_norm, DATASET, split='train', encode_categoricals=True)
test_fused = fusion.fuse(test_norm, DATASET, split='test', encode_categoricals=True)
bi_cols = fusion.get_bi_columns(train_fused)

bi_cont = [c for c in CONTINUOUS_BI_VARS if c in train_fused.columns]
bi_norm = DataNormalizer(method='minmax')
train_fused = bi_norm.fit_transform(train_fused, bi_cont)
test_fused = bi_norm.transform(test_fused)

print(f"Fused: {train_fused.shape} train, {test_fused.shape} test")

In [ ]:
# Build datasets for all 3 feature selection strategies
X_train_dict = {}
y_train_dict = {}
train_df_dict = {}
feature_names_dict = {}
X_test_dict = {}
y_test_dict = {}
test_df_dict = {}

# --- Correlation ---
sel_corr = BIAwareFeatureSelector(variance_threshold=0.01, correlation_threshold=0.95)
fn_corr = sel_corr.select_features(
    data=train_fused, sensor_cols=sensor_cols,
    bi_cols=bi_cols, setting_cols=setting_cols, exclude_cols=meta_cols)
tr_corr = sel_corr.transform(train_fused, keep_cols=meta_cols)
te_corr = sel_corr.transform(test_fused, keep_cols=meta_cols)
X_tr_c, y_tr_c = create_sliding_windows(tr_corr, W, fn_corr, 'rul', pad=PAD)
X_te_c, y_te_c = create_sliding_windows(te_corr, W, fn_corr, 'rul', pad=PAD)
X_train_dict['correlation'] = X_tr_c
y_train_dict['correlation'] = y_tr_c
train_df_dict['correlation'] = tr_corr
feature_names_dict['correlation'] = fn_corr
X_test_dict['correlation'] = X_te_c
y_test_dict['correlation'] = y_te_c
test_df_dict['correlation'] = te_corr

# --- AFICv ---
sel_aficv = AFICvFeatureSelector(base_learner='xgboost', n_folds=5, cumulative_threshold=0.90)
fn_aficv = sel_aficv.select_features_stratified(
    data=train_fused, sensor_cols=sensor_cols,
    bi_cols=bi_cols, setting_cols=setting_cols,
    target_col='rul', group_col='unit')
tr_aficv = sel_aficv.transform(train_fused, keep_cols=meta_cols)
te_aficv = sel_aficv.transform(test_fused, keep_cols=meta_cols)
X_tr_a, y_tr_a = create_sliding_windows(tr_aficv, W, fn_aficv, 'rul', pad=PAD)
X_te_a, y_te_a = create_sliding_windows(te_aficv, W, fn_aficv, 'rul', pad=PAD)
X_train_dict['aficv'] = X_tr_a
y_train_dict['aficv'] = y_tr_a
train_df_dict['aficv'] = tr_aficv
feature_names_dict['aficv'] = fn_aficv
X_test_dict['aficv'] = X_te_a
y_test_dict['aficv'] = y_te_a
test_df_dict['aficv'] = te_aficv

# --- Sensor only ---
fn_sensor = [f for f in fn_corr if f.startswith('sensor_') or f.startswith('setting_')]
tr_sensor = train_fused[meta_cols + fn_sensor].copy()
te_sensor = test_fused[meta_cols + fn_sensor].copy()
X_tr_s, y_tr_s = create_sliding_windows(tr_sensor, W, fn_sensor, 'rul', pad=PAD)
X_te_s, y_te_s = create_sliding_windows(te_sensor, W, fn_sensor, 'rul', pad=PAD)
X_train_dict['sensor_only'] = X_tr_s
y_train_dict['sensor_only'] = y_tr_s
train_df_dict['sensor_only'] = tr_sensor
feature_names_dict['sensor_only'] = fn_sensor
X_test_dict['sensor_only'] = X_te_s
y_test_dict['sensor_only'] = y_te_s
test_df_dict['sensor_only'] = te_sensor

print(f"\n=== Data ready ===")
for k in X_train_dict:
    print(f"  {k:15s}: {X_train_dict[k].shape[2]:2d} features, "
          f"train={X_train_dict[k].shape[0]}, test={X_test_dict[k].shape[0]}")

---
## 2. Stage 1: ML Branch GA (XGBoost)
Expected time: ~30-60 min on CPU

In [ ]:
save_dir = str(project_root / 'results' / 'ga' / DATASET)

ml_results = run_ml_ga(
    X_train_dict=X_train_dict,
    y_train_dict=y_train_dict,
    train_df_dict=train_df_dict,
    feature_names_dict=feature_names_dict,
    window_size=W,
    pad=PAD,
    pop_size=20,
    n_generations=30,
    save_dir=save_dir,
)

In [ ]:
# GA convergence plot
history = ml_results['ga'].get_history_df()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history['generation'], history['best_fitness'], 'o-', label='Gen best', alpha=0.5)
ax.plot(history['generation'], history['global_best'], 's-', label='Global best', linewidth=2)
ax.plot(history['generation'], history['mean_fitness'], '--', label='Gen mean', alpha=0.5)
ax.set_xlabel('Generation')
ax.set_ylabel('RMSE (validation, per-unit)')
ax.set_title(f'ML GA Convergence — {DATASET} ({FEATURE_SELECTION})', fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

print(f"\nBest params: {ml_results['best_params']}")
print(f"Best val RMSE: {ml_results['best_rmse']:.2f}")

In [ ]:
# Retrain best ML model on full training data and evaluate on test
from ml_branch import MLBranch

best_ml_params = ml_results['best_params']
best_fs_ml = best_ml_params['feature_selection']
print(f"Best feature selection: {best_fs_ml}")
print(f"Best params: {best_ml_params}")

ml_best = MLBranch(
    model_type='xgboost',
    flatten_strategy=best_ml_params['flatten_strategy'],
    n_estimators=best_ml_params['n_estimators'],
    max_depth=best_ml_params['max_depth'],
    learning_rate=best_ml_params['learning_rate_xgb'],
    subsample=best_ml_params['subsample'],
    colsample_bytree=best_ml_params['colsample_bytree'],
    reg_alpha=best_ml_params['reg_alpha'],
    reg_lambda=best_ml_params['reg_lambda'],
)
ml_best.fit(X_train_dict[best_fs_ml], y_train_dict[best_fs_ml],
            feature_names=feature_names_dict[best_fs_ml])

y_pred_ml = ml_best.predict(X_test_dict[best_fs_ml])

print(f"\n=== ML (GA-optimized) on TEST — {best_fs_ml} ===")
results_ml_ga = evaluate_per_unit(
    y_true=y_test_dict[best_fs_ml], y_pred=y_pred_ml,
    df=test_df_dict[best_fs_ml], window_size=W, pad=PAD,
)

---
## 3. Stage 2: DL Branch GA (BiLSTM + Attention)
Expected time: ~6-12h on CPU. **Run overnight.**

In [ ]:
dl_results = run_dl_ga(
    X_train_dict=X_train_dict,
    y_train_dict=y_train_dict,
    train_df_dict=train_df_dict,
    feature_names_dict=feature_names_dict,
    window_size=W,
    pad=PAD,
    pop_size=20,
    n_generations=30,
    max_epochs=50,
    save_dir=save_dir,
)

In [ ]:
# DL GA convergence plot
history_dl = dl_results['ga'].get_history_df()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history_dl['generation'], history_dl['best_fitness'], 'o-', label='Gen best', alpha=0.5)
ax.plot(history_dl['generation'], history_dl['global_best'], 's-', label='Global best', linewidth=2)
ax.plot(history_dl['generation'], history_dl['mean_fitness'], '--', label='Gen mean', alpha=0.5)
ax.set_xlabel('Generation')
ax.set_ylabel('RMSE (validation, per-unit)')
ax.set_title(f'DL GA Convergence — {DATASET} ({FEATURE_SELECTION})', fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

print(f"\nBest params: {dl_results['best_params']}")
print(f"Best val RMSE: {dl_results['best_rmse']:.2f}")

In [ ]:
# Retrain best DL model on full training data
import tensorflow as tf
from attention import build_dual_attention_bilstm, extract_attention_weights, save_attention_weights

best_dl_params = dl_results['best_params']
best_fs_dl = best_dl_params['feature_selection']
n_features_dl = X_train_dict[best_fs_dl].shape[2]
print(f"Best feature selection: {best_fs_dl} ({n_features_dl} features)")
print(f"Best params: {best_dl_params}")

model_best, attn_model_best = build_dual_attention_bilstm(
    window_size=W,
    n_features=n_features_dl,
    lstm_units=best_dl_params['lstm_units'],
    feature_attention_dim=best_dl_params['feature_attention_dim'],
    temporal_attention_dim=best_dl_params['temporal_attention_dim'],
    dropout_rate=best_dl_params['dropout_rate'],
    dense_units=best_dl_params['dense_units'],
    learning_rate=best_dl_params['learning_rate'],
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
]

history_retrain = model_best.fit(
    X_train_dict[best_fs_dl], y_train_dict[best_fs_dl],
    epochs=100,
    batch_size=best_dl_params['batch_size'],
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1,
)

y_pred_dl = model_best.predict(X_test_dict[best_fs_dl], batch_size=256).flatten()

print(f"\n=== DL (GA-optimized) on TEST — {best_fs_dl} ===")
results_dl_ga = evaluate_per_unit(
    y_true=y_test_dict[best_fs_dl], y_pred=y_pred_dl,
    df=test_df_dict[best_fs_dl], window_size=W, pad=PAD,
)

---
## 4. Hybrid Fusion (α optimization)

In [ ]:
from ml_branch import HybridPredictor

# Hybrid only works if DL and ML used same feature selection
if best_fs_ml == best_fs_dl:
    hybrid = HybridPredictor()
    hybrid.optimize_alpha(y_pred_dl, y_pred_ml, y_test_dict[best_fs_dl], metric='rmse')

    y_pred_hybrid = hybrid.predict(y_pred_dl, y_pred_ml)

    print(f"\n=== Hybrid (α={hybrid.alpha:.2f}) on TEST — {best_fs_dl} ===")
    results_hybrid = evaluate_per_unit(
        y_true=y_test_dict[best_fs_dl], y_pred=y_pred_hybrid,
        df=test_df_dict[best_fs_dl], window_size=W, pad=PAD,
    )
else:
    print(f"DL ({best_fs_dl}) and ML ({best_fs_ml}) used different feature selections.")
    print("Hybrid fusion requires retraining one branch with the other's features.")
    print("Skipping hybrid — report DL and ML results separately.")
    results_hybrid = None

---
## 5. Summary Table

In [ ]:
rows = [
    {'Model': f'XGBoost GA ({best_fs_ml})', 'RMSE': results_ml_ga['rmse_last'],
     'MAE': results_ml_ga['mae_last'], 'Score': results_ml_ga['score_last']},
    {'Model': f'BiLSTM+Attn GA ({best_fs_dl})', 'RMSE': results_dl_ga['rmse_last'],
     'MAE': results_dl_ga['mae_last'], 'Score': results_dl_ga['score_last']},
]
if results_hybrid is not None:
    rows.append({'Model': f'Hybrid (α={hybrid.alpha:.2f})', 'RMSE': results_hybrid['rmse_last'],
     'MAE': results_hybrid['mae_last'], 'Score': results_hybrid['score_last']})

summary = pd.DataFrame(rows)

print(f"\n{'='*60}")
print(f"FINAL RESULTS — {DATASET}")
print(f"{'='*60}")
print(summary.to_string(index=False))

In [ ]:
# Save everything
results_dir = project_root / 'results' / 'ga' / DATASET
results_dir.mkdir(parents=True, exist_ok=True)

summary.to_csv(results_dir / 'final_results.csv', index=False)

model_best.save(results_dir / f'best_dl_model_{best_fs_dl}.keras')

weights = extract_attention_weights(
    attn_model_best, X_test_dict[best_fs_dl], feature_names_dict[best_fs_dl])
save_attention_weights(weights, str(results_dir / 'attention_weights'),
                       dataset_name=f'M1_{best_fs_dl}_GA')

print(f"\nAll results saved to {results_dir}")

In [ ]:
# Download results
from google.colab import files
import shutil
shutil.make_archive('/content/ga_results', 'zip', str(project_root / 'results'))
files.download('/content/ga_results.zip')

---
## 6. Workflow

1. **Run Stage 1 (ML GA)** — ~30-60 min
2. **Run Stage 2 (DL GA)** — ~6-12h (run overnight)
3. **Run Hybrid fusion** — instant
4. **Repeat** with `FEATURE_SELECTION = 'aficv'` and `'sensor_only'`

### Files saved:
```
results/ga/FD001_correlation/
    ga_ml_best_params.json
    ga_ml_history.csv
    ga_dl_best_params.json
    ga_dl_history.csv
    final_results.csv
    best_dl_model.keras
    attention_weights/
```